# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
import os
import numpy as np
from collections import deque
from unityagents import UnityEnvironment

import matplotlib.pyplot as plt
%matplotlib inline

# from deterministic_dpg_agent import Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [2]:
env = UnityEnvironment(file_name="Reacher.app")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 1
Size of each action: 4
There are 1 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# states = env_info.vector_observations                  # get the current state (for each agent)
# scores = np.zeros(num_agents)                          # initialize the score (for each agent)
# while True:
#     actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
#     actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
#     env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#     next_states = env_info.vector_observations         # get next state (for each agent)
#     rewards = env_info.rewards                         # get reward (for each agent)
#     dones = env_info.local_done                        # see if episode finished
#     scores += env_info.rewards                         # update the score (for each agent)
#     states = next_states                               # roll over states to next time step
#     if np.any(dones):                                  # exit loop if episode finished
#         break
# print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

# UTILS.py

In [6]:
import numpy as np
import random
import copy
from collections import namedtuple, deque

import torch


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed, device):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
        self.device = device
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(self.device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)
    
class OrnsteinUhlenbeckProcess:
    def __init__(self, size, seed, std=.2, theta=.15, dt=1e-2, x0=None):
        self.theta = theta
        self.mu = 0
        self.std = std
        self.dt = dt
        self.x0 = x0
        self.size = size
        self.seed = random.seed(seed)
        self.reset()

    def sample(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev) * self.dt + self.std * np.sqrt(
            self.dt) * np.random.randn(*[self.size])
        self.x_prev = x
        return x

    def reset(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros(self.size)

# MODEL.py

In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from collections import OrderedDict


def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

class DDPGActor(nn.Module):
    def __init__(self, obs_size, act_size, fc1_units=400, fc2_units=300):
        super(DDPGActor, self).__init__()

        self.net = nn.Sequential(OrderedDict([
            ('fc1', nn.Linear(obs_size, fc1_units)),
            ('bn1', nn.BatchNorm1d(fc1_units)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(fc1_units, fc2_units)),
            ('bn2', nn.BatchNorm1d(fc2_units)),
            ('relu2', nn.ReLU()),
            ('fc3', nn.Linear(fc2_units, act_size)),
            ('tanh3', nn.Tanh())
        ]))
        self.reset_parameters()

    def forward(self, x):
        return self.net(x)
    
    def reset_parameters(self):
        self.net.fc1.weight.data.uniform_(*hidden_init(self.net.fc1))
        self.net.fc2.weight.data.uniform_(*hidden_init(self.net.fc2))
        self.net.fc3.weight.data.uniform_(-3e-3, 3e-3)


class DDPGCritic(nn.Module):
    def __init__(self, obs_size, act_size, fc1_units=400, fc2_units=300):
        super(DDPGCritic, self).__init__()

        self.obs_net = nn.Sequential(OrderedDict([
             ('fcs1', nn.Linear(obs_size, fc1_units)),
             ('bn1', nn.BatchNorm1d(fc1_units)),
             ('relu1', nn.ReLU()),
        ]))

        self.out_net = nn.Sequential(OrderedDict([
            ('fc2', nn.Linear(fc1_units + act_size, fc2_units)),
            ('bn2', nn.BatchNorm1d(fc2_units)),
            ('relu2', nn.ReLU()),
            ('fc3', nn.Linear(fc2_units, 1))
        ]))
        self.reset_parameters()

    def forward(self, states, actions):
        obs = self.obs_net(states)
        return self.out_net(torch.cat([obs, actions], dim=1))
    
    def reset_parameters(self):
        self.obs_net.fcs1.weight.data.uniform_(*hidden_init(self.obs_net.fcs1))
        self.out_net.fc2.weight.data.uniform_(*hidden_init(self.out_net.fc2))
        self.out_net.fc3.weight.data.uniform_(-3e-3, 3e-3)


# Agent

In [8]:
import numpy as np
import random
from collections import namedtuple, deque

# from model import DDPGActor, DDPGCritic
# from utils import ReplayBuffer, OrnsteinUhlenbeckProcess

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e6)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
REPLAY_INITIAL = int(1e4) # initial memory before updatting the network
GAMMA = 0.99            # discount factor
LR_ACTOR = 1e-4         # learning rate 
LR_CRITIC = 1e-3        # learning rate of the critic
UPDATE_EVERY = 4        # how often to update the network
TAU = 1e-3              # soft update
WEIGHT_DECAY = 0        # L2 weight decay
NET_BODY = (400, 300)   # hidden layers

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self,
                 state_size,
                 action_size,
                 seed,
                 net_body=NET_BODY,
                 gamma=GAMMA,
                 buffer_size=BUFFER_SIZE,
                 batch_size=BATCH_SIZE,
                 replay_initial=REPLAY_INITIAL,
                 update_every=UPDATE_EVERY,
                 lr_actor=LR_ACTOR,
                 lr_critic=LR_CRITIC,
                 weight_decay=WEIGHT_DECAY,
                 tau=TAU
    ):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)
        self.gamma = gamma
        self.batch_size = batch_size
        self.replay_initial = replay_initial
        self.update_every = update_every
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Actor-Network
        fc1_units, fc2_units = net_body
        self.actor_local = DDPGActor(state_size, action_size, fc1_units, fc2_units).to(self.device)
        self.actor_target = DDPGActor(state_size, action_size, fc1_units, fc2_units).to(self.device)
        self.actor_optimizer = optim.Adam(self.actor_local.parameters(), lr=lr_actor)
        
        # Critic-Network
        self.critic_local = DDPGCritic(state_size, action_size, fc1_units, fc2_units).to(self.device)
        self.critic_target = DDPGCritic(state_size, action_size, fc1_units, fc2_units).to(self.device)
        self.critic_optimizer = optim.Adam(self.critic_local.parameters(), lr=lr_critic, weight_decay=weight_decay)
    
        # Noise process
        self.noise = OrnsteinUhlenbeckProcess(action_size, self.seed)
        
        # Replay memory
        self.memory = ReplayBuffer(
            action_size=action_size,
            buffer_size=buffer_size,
            batch_size=batch_size,
            seed=seed,
            device=self.device
        )
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        """Save experience in replay memory, and use random sample from buffer to learn."""
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % self.update_every
        # If enough samples are available in memory, get random subset and learn
        if (len(self.memory) > self.replay_initial) and (self.t_step == 0):
            experiences = self.memory.sample()
            self.update(experiences, self.gamma)

    def act(self, state, add_noise=True):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
        """
        state = (
            torch.from_numpy(state)
            .float()
            .to(self.device)
        )
        
        self.actor_local.eval()
        with torch.no_grad():
            action = self.actor_local.forward(state).cpu().data.numpy()
        self.actor_local.train()
        if add_noise:
            action += self.noise.sample()
        return np.clip(action, -1., 1.)
    
    def reset(self):
        self.noise.reset()

    def update(self, batch, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            batch (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = batch
        
        # Update Critic
        # Get expected Q values from critic model
        curr_Q = self.critic_local.forward(states, actions)
        
        # Get predicted Q values (for next states) from target model
        next_actions = self.actor_target.forward(next_states)
        next_Q = self.critic_target.forward(next_states, next_actions)
        expected_Q = rewards + (self.gamma * next_Q * (1 - dones))

        critic_loss = F.mse_loss(curr_Q, expected_Q)

        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()
        
        # Update Actor
        # Get expected actions from actor model
        actions_pred = self.actor_local(states)
        actor_loss = -self.critic_local(states, actions_pred).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        # Update target networks
        self.update_target(self.critic_local, self.critic_target, TAU)  
        self.update_target(self.actor_local, self.actor_target, TAU) 
        
    def update_target(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

# Main.py

In [9]:
def train(agent, env, n_episodes=2000, max_t=1000, min_score=30, num_agents=1):
    """DDPG-Learning.
    Params
    ======
        agent: An instance of Agent class
        env: Unity environment
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    score_list = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        for t in range(max_t):
            actions = agent.act(states)[0]
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations      # get the next state
            rewards = env_info.rewards                     # get the reward
            dones = env_info.local_done                    # see if episode has finished
            agent.step(states, actions, rewards, next_states, dones)
            scores += rewards                                # update the score
            states = next_states                             # roll over the state to next time step
            if all(dones):
                break
        mean_score = np.mean(scores)
        scores_window.append(mean_score)           # save most recent score
        score_list.append(mean_score)              # save most recent score
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
        if np.mean(scores_window) >= min_score:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            break
    return score_list

# agent = Agent(state_size=33, action_size=4, seed=0)
# scores = train(agent, env, n_episodes=1, num_agents=1)

In [10]:
# load the weights from file
def execute_policy(env, agent, num_agents):
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    states = env_info.vector_observations              # get the current state (for each agent)
    scores = np.zeros(num_agents)                      # initialize the score (for each agent)
    while True:
        actions = agent.act(states)                        # select an action (for each agent)
        env_info = env.step(actions)[brain_name]           # send the action to the environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))
    

def evaluate_agent(env, agent, checkpoint_pth='.', num_episodes=100, min_score=30):
    score_lst = []
    agent.actor_local.load_state_dict(torch.load(f'{checkpoint_pth}/checkpoint_actor.pth'))
    agent.critic_local.load_state_dict(torch.load(f'{checkpoint_pth}/checkpoint_critic.pth'))
    for i in range(num_episodes):
        score_lst.append(execute_policy(env, agent))

    assert np.mean(score_lst) >= min_score,\
        f'Environment not solved: Expected score >= {min_score}, found {np.mean(score_lst)}'

# Execute Training/Evaluate Agent

In [11]:
from datetime import datetime
from workspace_utils import active_session
start_time = datetime.now()

# VARS
config = {
    "buffer_size": int(1e5),    # replay buffer size
    "batch_size": 128,          # minibatch size
    "replay_initial": 128,      # initial memory before updating the network
    "gamma": 0.99,              # discount factor
    "lr_actor": 1e-4,           # learning rate 
    "lr_critic": 1e-3,          # learning rate of the critic
    "update_every": 4,          # how often to update the network
    "tau": 1e-3,                # soft update
    "weight_decay": 0,          # l2 weight decay
    "net_body": (256, 128),     # hidden layers    
}

agent = Agent(state_size=33, action_size=4, seed=0, **config)

# with active_session():
print("Train Agent")
scores = train(agent, env, n_episodes=10, max_t=1000)

# Plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

time_elapsed = datetime.now() - start_time 
print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Train Agent


KeyboardInterrupt: 

In [ ]:
# print("Evaluate Agent")
# evaluate_agent(env, agent, '.')

In [ ]:
env.close()